## Code written and working on simulated data set

In [83]:
import numpy as np
import scipy as sp

In [117]:
def PhiGamma(alpha, beta, w, T):
    k = len(alpha)
    N = len(w)
    alpha = np.array(alpha)
    beta = np.array(beta)
    gamma = np.tile(alpha + N/k, ((T+1),1))
    phi = np.array([[[1/k]*k]*N]*(T+1))
    for t in range(T):
        for n in range(N):
            for i in range(k):
                phi[t+1,n,i] = beta[i,n] * np.exp(sp.special.psi(gamma[t,i])) #beta[i,n], n should be w_n.
            phi_const = np.sum(phi[t+1,n,])
            phi[t+1,n,] = phi[t+1,n,]/phi_const
        gamma[t+1,] = alpha + np.sum(phi[t+1,], axis=0)
    return(gamma[T,],phi[T,])

In [121]:
#Test function
N=4
T=5
alpha = np.array([1/3,1/3,1/3])
beta = np.array([[1/2,1/2,1/2,1/2,1/2,1/2], [1/2,1/2,1/2,1/2,1/2,1/2], [1/2,1/2,1/2,1/2,1/2,1/2]])
w=0
a = np.stack((beta,beta))
PhiGamma(alpha, beta, w, N, T)

(array([ 1.66666667,  1.66666667,  1.66666667]),
 array([[ 0.33333333,  0.33333333,  0.33333333],
        [ 0.33333333,  0.33333333,  0.33333333],
        [ 0.33333333,  0.33333333,  0.33333333],
        [ 0.33333333,  0.33333333,  0.33333333]]))

In [ ]:
def e_step(self, parsed_corpus=None, local_parameter_iteration=50, local_parameter_converge_threshold=1e-6):
        if parsed_corpus == None:
            word_ids = self._parsed_corpus[0];
            word_cts = self._parsed_corpus[1];
        else:
            word_ids = parsed_corpus[0]
            word_cts = parsed_corpus[1];
        
        assert len(word_ids) == len(word_cts);
        number_of_documents = len(word_ids);
        
        document_log_likelihood = 0;
        words_log_likelihood = 0;

        # initialize a V-by-K matrix phi sufficient statistics
        phi_sufficient_statistics = np.zeros((self._number_of_topics, self._number_of_types));
        
        # initialize a D-by-K matrix gamma values
        gamma_values = np.zeros((number_of_documents, self._number_of_topics)) + self._alpha_alpha[np.newaxis, :] + 1.0 * self._number_of_types / self._number_of_topics;
        
        E_log_eta = compute_dirichlet_expectation(self._eta);
        assert E_log_eta.shape == (self._number_of_topics, self._number_of_types);
        if parsed_corpus != None:
            E_log_prob_eta = E_log_eta - sp.misc.logsumexp(E_log_eta, axis=1)[:, np.newaxis]
        
        # iterate over all documents
        # for doc_id in xrange(number_of_documents):
        for doc_id in np.random.permutation(number_of_documents):
            # compute the total number of words
            # total_word_count = self._corpus[doc_id].N()
            total_word_count = np.sum(word_cts[doc_id]);

            # initialize gamma for this document
            gamma_values[doc_id, :] = self._alpha_alpha + 1.0 * total_word_count / self._number_of_topics;
            
            # term_ids = np.array(self._corpus[doc_id].keys());
            # term_counts = np.array([self._corpus[doc_id].values()]);
            term_ids = word_ids[doc_id];
            term_counts = word_cts[doc_id];
            assert term_counts.shape == (1, len(term_ids));

            # update phi and gamma until gamma converges
            for gamma_iteration in xrange(local_parameter_iteration):
                assert E_log_eta.shape == (self._number_of_topics, self._number_of_types);
                log_phi = E_log_eta[:, term_ids].T + np.tile(sp.special.psi(gamma_values[[doc_id], :]), (word_ids[doc_id].shape[0], 1));
                assert log_phi.shape == (len(term_ids), self._number_of_topics);
                log_phi -= sp.misc.logsumexp(log_phi, axis=1)[:, np.newaxis];
                assert log_phi.shape == (len(term_ids), self._number_of_topics);
                
                gamma_update = self._alpha_alpha + np.array(np.sum(np.exp(log_phi + np.log(term_counts.transpose())), axis=0));
                
                mean_change = np.mean(abs(gamma_update - gamma_values[doc_id, :]));
                gamma_values[doc_id, :] = gamma_update;
                if mean_change <= local_parameter_converge_threshold:
                    break;
            
            # Note: all terms including E_q[p(\theta | \alpha)], i.e., terms involving \Psi(\gamma), are cancelled due to \gamma updates in E-step
            
            # compute the alpha terms
            document_log_likelihood += sp.special.gammaln(np.sum(self._alpha_alpha)) - np.sum(sp.special.gammaln(self._alpha_alpha))
            # compute the gamma terms
            document_log_likelihood += np.sum(sp.special.gammaln(gamma_values[doc_id, :])) - sp.special.gammaln(np.sum(gamma_values[doc_id, :]));
            # compute the phi terms
            document_log_likelihood -= np.sum(np.dot(term_counts, np.exp(log_phi) * log_phi));
            
            # Note: all terms including E_q[p(\eta | \beta)], i.e., terms involving \Psi(\eta), are cancelled due to \eta updates in M-step
            if parsed_corpus != None:
                # compute the p(w_{dn} | z_{dn}, \eta) terms, which will be cancelled during M-step during training
                words_log_likelihood += np.sum(np.exp(log_phi.T + np.log(term_counts)) * E_log_prob_eta[:, term_ids]);
            
            assert(log_phi.shape == (len(term_ids), self._number_of_topics));
            phi_sufficient_statistics[:, term_ids] += np.exp(log_phi + np.log(term_counts.transpose())).T;
            
            if (doc_id + 1) % 1000 == 0:
                print "successfully processed %d documents..." % (doc_id + 1);
        
        if parsed_corpus == None:
            self._gamma = gamma_values;
            return document_log_likelihood, phi_sufficient_statistics
        else:
            return words_log_likelihood, gamma_values

In [ ]:
def m_step(self, phi_sufficient_statistics):
        # Note: all terms including E_q[p(\eta|\beta)], i.e., terms involving \Psi(\eta), are cancelled due to \eta updates
            
        # compute the beta terms
        topic_log_likelihood = self._number_of_topics * (sp.special.gammaln(np.sum(self._alpha_beta)) - np.sum(sp.special.gammaln(self._alpha_beta)));
        # compute the eta terms
        topic_log_likelihood += np.sum(np.sum(sp.special.gammaln(self._eta), axis=1) - sp.special.gammaln(np.sum(self._eta, axis=1)));
        
        self._eta = phi_sufficient_statistics + self._alpha_beta;
        assert(self._eta.shape == (self._number_of_topics, self._number_of_types));
        
        # self._E_log_eta = compute_dirichlet_expectation(self._eta);
        
        # compute the sufficient statistics for alpha and update
        alpha_sufficient_statistics = sp.special.psi(self._gamma) - sp.special.psi(np.sum(self._gamma, axis=1)[:, np.newaxis]);
        alpha_sufficient_statistics = np.sum(alpha_sufficient_statistics, axis=0);  # [np.newaxis, :];
        
        return topic_log_likelihood, alpha_sufficient_statistics

## 1. Background Introduction
### A. Question interested
Due to the Internet, information retrieval has exploded. For a range of reasons in the modern world, people want to compare and contrast two documents for structure, text, words, topics. The goal is to retrieve information from a large text corpus. “Thus each word is generated from a single topic, and different words in one document can be generated by different topics. Each document is represented as a list of mixing proportions for these mixture components and thereby reduced to a probability distribution on a fixed set of topics. This distribution is the ‘reduced description’ associated with the document”. 
### B. Basic thoughts and procedure
A significant step forward in this regard was made by modeling each word in a document as a sample from a mixture model, where the mixture components are random variables that can be viewed as representations of “topics”. The solution to the goal is the Latent Dirichlet Allocation (LDA) model, a three-level probabilistic model in natural language processing for comparing and contrasting collections of discrete data, and providing a short description of the statistical relationships in those large discrete data sets. In LDA, each document is composed of various topics with Dirichlet prior distribution. Each topic has a probability of generating its corresponding word and those words that do not belong to any topic have an even probability of being placed into each category.



## 2. Algorithm 
### A. Models
#### (1). Notation
Formally, we define the following terms:
- A _word_ is the basic unit of discrete data, defined to be an item from a vocabulary indexed by {1, . . . , V }. We represent words using unit-basis vectors that have a single component equal to one and all other components equal to zero. Thus, using superscripts to denote components, the _v_th word in the vocabulary is represented by a V -vector w such that $w^v = 1$ and $w^u = 0$ for $u\neq v$.
- _A _document_ is a sequence of N words denoted by $\textbf{w} = (w_1,w_2,... ,w_N)$, where $w_n$ is the _n_ th word in the sequence.
- A _corpus_ is a collection of M documents denoted by $D = \{\textbf{w}_1,\textbf{w}_2,... ,\textbf{w}_M \}$.

#### (2). Latent Dirichlet allocation
LDA assumes the following generative process for each document $\textbf{w}$ in a corpus D:
1. Choose N $\sim$ Poisson(ξ).
2. Chooseθ $\sim$ Dir(α).
3. For each of the N words $w_n$:
 - Choose a topic $z_n$ $\sim$ Multinomial(θ).
 - Choose a word $w_n$ from p($w_n$ |$z_n$,β), a multinomial probability conditioned on the topic $z_n$.

### B. Techniques for inference
#### (1). Variational inference

#### (2). Parameter estimation
The derivation of the variational EM algorithm for LDA yields the following iterative algorithm:
  1. (E-step) For each document, find the optimizing values of the variational parameters {$γ^*_d , φ^*_d : d \in D$}. This is done as described in the previous section.
  2. (M-step) Maximize the resulting lower bound on the log likelihood with respect to the model parameters α and β. This corresponds to finding maximum likelihood estimates with expected sufficient statistics for each document under the approximate posterior which is computed in the E-step.

## 3. Application
### A. Simulated data
### B. Real dataset
## 4. Optimization and Analysis
## 5. Discussion